In [ ]:
!nvidia-smi

Wed Dec 28 07:26:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    33W /  70W |    988MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install  yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
data = yf.download("TCS.NS" , start = "2010-01-10" , interval = '1d')

[*********************100%***********************]  1 of 1 completed


In [ ]:
data.shape

(3202, 6)

In [ ]:
data.head(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-11,352.0,360.049988,350.200012,357.100006,280.941132,4318408
2010-01-12,366.0,377.000000,363.500000,375.024994,295.043182,9027690
2010-01-13,372.5,398.200012,372.325012,394.274994,310.187714,10910272


Understanding Trends with in the Data

In [ ]:
#Sort the data points based on indexes just for confirmation
data.sort_index(inplace = True)

In [ ]:
#Remove any duplicate index
data =data.loc[~data.index.duplicated(keep='first')]

In [ ]:
data.tail(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-12-26,3228.350098,3272.000000,3225.050049,3252.899902,3252.899902,870157
2022-12-27,3269.199951,3273.800049,3231.500000,3259.500000,3259.500000,835883
2022-12-28,3249.800049,3253.000000,3226.000000,3252.000000,3252.000000,437676


In [ ]:
#Check for misssing values
data.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
#Get the statistics of the data
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,3202.000000,3202.000000,3202.000000,3202.000000,3202.000000,3.202000e+03
mean,1561.037007,1577.169969,1543.738878,1560.352568,1447.971521,2.833583e+06
std,958.711090,966.944781,949.107481,957.882361,982.762042,2.300830e+06
min,352.000000,357.000000,342.625000,349.774994,275.908051,0.000000e+00
25%,730.137497,739.062500,724.381241,733.937515,607.070862,1.747309e+06
50%,1260.000000,1272.174988,1247.262512,1259.000000,1112.745300,2.372553e+06
75%,2100.000000,2123.924988,2073.875000,2101.474915,1983.169006,3.338434e+06
max,4033.949951,4043.000000,3980.000000,4019.149902,3964.502930,8.806715e+07


In [ ]:
import plotly.graph_objects as go

#Check the trend in closing values
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Close'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900 ,
                  xaxis_title = 'Date' , yaxis_title='Close')
fig.show()

Data Preparation

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pickle
from tqdm.notebook import tnrange

In [ ]:
#Filter only required data
data = data [['Close' , 'Volume']]
data.head(3)

,Close,Volume
Date,,
2010-01-11,357.100006,4318408
2010-01-12,375.024994,9027690
2010-01-13,394.274994,10910272


In [ ]:
#confirm the testing set length
test_length = data[(data.index >= '2020-01-10')].shape[0]

In [ ]:
def CreateFeatures_and_Targets(data, feature_length):
  X = []
  Y = []

  for i in tnrange(len(data) - feature_length):
    X.append(data.iloc[i : i + feature_length,:].values)
    Y.append(data["Close"].values[i + feature_length])

  X = np.array(X)
  Y = np.array(Y)

  return X , Y

In [ ]:
X , Y = CreateFeatures_and_Targets(data , 32)

  0%|          | 0/3170 [00:00<?, ?it/s]

In [ ]:
#Check the shapes
X.shape , Y.shape

((3170, 32, 2), (3170,))

In [ ]:
Xtrain , Xtest , Ytrain , Ytest = X[:-test_length] , X[-test_length:] , Y[:-test_length] , Y[-test_length:]

In [ ]:
#Check the Training Dataset Shape
Xtrain.shape , Ytrain.shape

((2432, 32, 2), (2432,))

In [ ]:
#Check the Testing Dataset Shape
Xtest.shape , Ytest.shape

((738, 32, 2), (738,))

In [ ]:
#Create a Scaler to Scale Vector with multiple dimensions
class MultiDimensionScaler():
  def __init__(self):
    self.scalers = []

  def fit_transform(self , X):
    total_dims = X.shape[2]
    for i in range(total_dims):
      Scaler = MinMaxScaler()
      X[:, :, i] = Scaler.fit_transform(X[:, :, i])
      self.scalers.append(Scaler)
    return X

  def transform(self ,X):
    for i in range(X.shape[2]):
      X[:, :, i] = self.scalers[i].transform(X[:,:,i])
    return X 

In [ ]:
Feature_Scaler = MultiDimensionScaler()
Xtrain = Feature_Scaler.fit_transform(Xtrain)
Xtest = Feature_Scaler.transform(Xtest)

In [ ]:
Target_Scaler = MinMaxScaler()
Ytrain = Target_Scaler.fit_transform(Ytrain.reshape(-1,1))
Ytest = Target_Scaler.transform(Ytest.reshape(-1,1))

In [ ]:
def save_object(obj , name : str):
  pickle_out = open(f"{name}.pck","wb")
  pickle.dump(obj, pickle_out)
  pickle_out.close()

def load_object(name : str):
  pickle_in = open(f"{name},pck","rb")
  data = pickle.load(pickle_in)
  return data

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint , ReduceLROnPlateau

save_best = ModelCheckpoint("best_weights.h5", monitor='val_loss', save_best_only=True, save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25,patience=5, min_lr=0.00001, verbose= 1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , LSTM , Bidirectional

model = Sequential()

model.add(Bidirectional(LSTM(512 , return_sequences=True , recurrent_dropout=0.1, input_shape=(32, 2))))
model.add(LSTM(256 , recurrent_dropout=0.1))
model.add(Dropout(0.3))
model.add(Dense(64 , activation='elu'))
model.add(Dropout(0.3))
model.add(Dense(32 , activation='elu'))
model.add(Dense(1 , activation='linear'))  #Final Layer

In [ ]:
#optimizer = tf.keras.optimizers.Adam(learning_rate=0.002)
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.002)
model.compile(loss='mse', optimizer=optimizer)

In [ ]:
history = model.fit(Xtrain, Ytrain,
                    epochs=10,
                    batch_size=1,
                    verbose=1,
                    shuffle=False,
                    validation_data=(Xtest , Ytest),
                    callbacks=[reduce_lr , save_best])

Epoch 1/10
2432/2432 [==============================] - 367s 149ms/step - loss: 0.0028 - val_loss: 0.2486 - lr: 0.0020
Epoch 2/10
2432/2432 [==============================] - 359s 148ms/step - loss: 0.0043 - val_loss: 0.1714 - lr: 0.0020
Epoch 3/10
2432/2432 [==============================] - 367s 151ms/step - loss: 0.0035 - val_loss: 0.0973 - lr: 0.0020
Epoch 4/10
2432/2432 [==============================] - 375s 154ms/step - loss: 0.0031 - val_loss: 0.0935 - lr: 0.0020
Epoch 5/10
2432/2432 [==============================] - 385s 158ms/step - loss: 0.0026 - val_loss: 0.0639 - lr: 0.0020
Epoch 6/10
2432/2432 [==============================] - 382s 157ms/step - loss: 0.0022 - val_loss: 0.0567 - lr: 0.0020
Epoch 7/10
2432/2432 [==============================] - 385s 158ms/step - loss: 0.0020 - val_loss: 0.0386 - lr: 0.0020
Epoch 8/10
2432/2432 [==============================] - 381s 157ms/step - loss: 0.0019 - val_loss: 0.0375 - lr: 0.0020
Epoch 9/10
2432/2432 [==========================

In [ ]:
# Load the best weights
#model.load_weights("best_weights.h5")

Visualize the prediction on Test Set

In [ ]:
Predictions = model.predict(Xtest)

24/24 [==============================] - 2s 45ms/step


In [ ]:
Predictions= Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Ytest)

In [ ]:
Predictions.shape

(738, 1)

In [ ]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [ ]:
# Check the Predictions vs Actual
fig= go.Figure()

fig.add_trace(go.Scatter(x= data.index[-test_length:] , y= Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x= data.index[-test_length:] , y= Predictions , mode='lines' , name='Predicted'))
fig.show()

In [ ]:
Total_features = np.concatenate((Xtrain , Xtest) , axis=0)

In [ ]:
Total_Targets = np.concatenate((Ytrain , Ytest) , axis=0)

In [ ]:
Predictions = model.predict(Total_features)

100/100 [==============================] - 4s 39ms/step


In [ ]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Total_Targets)

In [ ]:
Predictions = np.squeeze(Predictions , axis=1)
Actual = np.squeeze(Actual , axis=1)

In [ ]:
#check the trend in Volume Traded
fig = go.Figure()
fig.add_trace(go.Scatter(x= data.index , y= Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x= data.index , y= Predictions , mode ='lines' , name='Predicted'))
fig.show()

In [ ]:
import requests

response = requests.get('https://www.alphavantage.co/query?function=RSI&symbol=TCS.NS&interval=daily&time_period=5&series_type=close&apikey=WG5TRKKPO7B8LUZ9')
response = response.json()

In [ ]:
response.keys()

dict_keys([])

In [ ]:
rsi_data = pd.DataFrame.from_dict(response['Technical Analysis:RSI'] , orient='index') 

KeyError: ignored

In [ ]:
rsi_data.head() 

In [ ]:
rsi_data = rsi_data[rsi_data.index >= '2020-03-01']

In [ ]:
rsi_data['RSI'] = rsi_data['RSI'].astype(np.float64)

In [ ]:
rsi_data.head()

In [ ]:
data = data.merge(rsi_data, left_index=True, right_index=True, how='inner')

In [ ]:
data.head()

RealTime Prediction

In [ ]:
#save and load the whole model
#model.save("load_weights.h5")
#loaded_model = tf.keras.model.load_model("load_weights.h5")

In [ ]:
#def PredictStockPrice(Model, Dataframe , PreviousDate , feature_length = 32):
  #idx_location = DataFrame.index.get_loc(PreviousDate)
  #Features = DataFrame.iloc[idx_location - feature_length : idx_location,:].values
  #Features = np.expand_dims(Features , axis = 0)
  #Features = Features_Scaler.transform(Features)
  #Prediction = Model.predict(Features)
  #Prediction = Target_Scaler.inverse_transform(Prediction)
  #return Prediction[0][0]

In [ ]:
#PredictstockPrice(loaded_model , data , '2020-03-01')